# Corona Dialog  mit Sprache und Emotion

### Elisa Seyfried, Praktische Anwendungen in Berufsfeldern: Dialogsystem, SS21

At the beginning of the script the neccesasary libraries are being imported

In [1]:
import requests
from eliza import eliza
import os
import json
from wendel_util import file_update
import dialog_util as util
import emorec


SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In the following cell the function file_update() is being accessed through the script wendel_util. The function checks if the file vaccinations.json already exists and if the last time stamp is from  today. If  it's an older version,  the newest data is downloaded from the api with the vaccination numbers. The output from the RKI is saved in the downloaded json-file and safed in the variable vaccinations.

In [2]:
file_update()
vaccinations = open('vaccinations.json')
vaccinations = json.load(vaccinations)

Web site exists
File Updated 


In the following cell the function init_google() in the python-file dialog-util is accessed.  It initializes the Google Automatic Speech Recognition by checking the credentials. These are available by registering on the Google  Cloud and downloading the credentials as json-file.

In [3]:
util.init_google()

In [4]:
def normalize(in_s):
    return in_s.lower()

In the following cell the funtion do_input() returns the results of the  function speech_input() that is in the dialog_util.py. The function speech_input calls the functions record_file() and transcribe(). record_file makes a recording of the user input and converts the recording to 16 bit integers. The function transcribe() accesses the recorded wav-file and uses the google automatic speech recognition to make a german transcript of the text.

In [5]:
def do_input():
    return util.speech_input()

In the following cell the dictionaries states_d and vaccines_d in dialog_util are accessed. It finds the keys in the input  through comparing them with the keys in the dictionary. When a keyword is found, the loop stops.

In [6]:
def semantic(input_s):
    semantics = {'state':'', 'vaccine':'', 'answer':0} #data structure
    for key in util.states_d.keys():
        if key in input_s:
            semantics['state'] = util.states_d[key]
    for key in util.vaccines_d.keys():
        if key in input_s:
            semantics['vaccine'] = util.vaccines_d[key]
    return semantics

In the following cell the data from the RKI API is accessed and the required information is extracted. It fills the answer, gets the input from RKI and fills the semantic object

In [7]:
def data(semantics):
    s = semantics['state']
    v = semantics['vaccine']
    if s: #state given
        if s != 'DE':
            if v: #and vaccine given
                semantics['answer'] = vaccinations["data"]["states"][s]["vaccination"][v] 
            else: #all vaccines for state
                semantics['answer'] = vaccinations["data"]["states"][s]["vaccinated"]
        else:
            if v: #and vaccine given
                semantics['answer'] = vaccinations["data"]["vaccination"][v]
            else: #all vaccines in Germany
                semantics['answer'] = vaccinations["data"]["vaccinated"]
    else: #no state
        if v: #but vaccine
            semantics['answer'] = vaccinations["data"]["vaccination"][v]
        else: #nothing given
            semantics['answer'] = None
    return semantics

The function in the following cell gives out the results and output string and returns it as text.

In [8]:
def output(semantics, inputs): #fügt gut oder schlecht ein
    ret = ''
    s = semantics['state']
    v = semantics['vaccine']
    a = semantics['answer']
    if s: #if state is given
        s = util.state_names[s]
        if v: #vaccine also given
            v = util.vaccine_names[v]
            ret = 'Die Impfungen für {} mit {} sind {}'.format(s, v, a) #formatting
        else: #all vaccines for states
            ret = 'Die Impfungen für {} sind {}'.format(s, a)
    else: #no state
        if v: #but vaccine
            v = util.vaccine_names[v]
            ret = 'Die Impfungen in Deutschland mit {} sind {}'.format(v, a)
        else: #nothing given
            #ret = 'Diese Frage kann ich leider nicht beantworten'
            ret = elz.respond(inputs) #output
    return ret
    
#def do_error():
 #   return 'Bitte wählen Sie ein Bundesland um die Coronazahlen zu erhalten'

In the following cell eliza is initialized by accessing the deutsch.txt in directory. The deutsch.txt contains all posssible utterances of eliza in German.

In [9]:
def init_eliza():
    root = "C:/Users/elisa/Seminare_Python/S_dialogsystem"
    elz = eliza.Eliza() #instance of eliza class
    elz.load(root+"/eliza/deutsch.txt") 
    return elz

The following cell initializes the speech synthesis. It prints the output and acesses the function tts(text) in dialog_util.py. The  Text-to-Speech used here is pytts from espeak. The voice, the language and the  speech rate can be chosen. 

In [10]:
#speech synthesis
def output_s(text):
    print('output: '+text)
    util.tts(text)

In the following dictionary the categories of the  emotion recognition are being defined in a dictionary to translate them into German for the output.

In [11]:
emo_dict = {'happiness': 'freundlich', 'neutral': 'neutral', 'anger': 'ärgerlich', 'sadness': 'traurig', 'disgust': 'angeekelt', 'fear': 'ängstlich', 'boredom': 'gelangweilt'}

In the following cell is the function dialogmanager(), which gives the output "Frag was" to initialize the dialog, then it accesses do_input() and gets the input there. In the next step  in normalizes the input. In a while-loop, which stops when input string "fertig" is inserted, the emotional recognizer is initiated and integrated in the output. Then semantics is defined which returns the data. Also the output and input are accessed.

In [12]:
def dialogmanager():
    output_s("Frag was!")
    input_s = do_input() #do_input() wird aufgerufen, um sich input zu holen
    input_s = normalize(input_s)
    while input_s and input_s != 'fertig': #wenn input-string 'fertig'  ist, wird Schleife nicht ausgeführt
        emotion = emoRec.classify(filename)[0]
        emotion_g = emo_dict[emotion]
        output_s('Ich merke du bist '+emotion_g)
        semantics = semantic(input_s) #funktion gibt etwas zurück
        semantics = data(semantics)
        out_string = output(semantics, input_s)
        output_s(out_string)
        input_s = do_input()
        input_s = normalize(input_s)

The following cell calls the functions init_eliza(), emorec.Emorec() and dialogmanager and starts the whole dialog.

In [14]:
filename = 'myfile.wav'
elz = init_eliza()
emoRec = emorec.EmoRec()
dialogmanager()



output: Frag was!
Transcript 0: fertig
